# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse   Shipments  \
0  Feb  3 2023 12:40PM  256623        12     7547722   
1  Feb  3 2023 12:35PM  256621        16     9111824   
2  Feb  3 2023 12:30PM  256620        10  0086354605   
3  Feb  3 2023 12:30PM  256620        10  0086354606   
4  Feb  3 2023 12:30PM  256620        10  0086354602   
5  Feb  3 2023 12:30PM  256620        10  0086354603   
6  Feb  3 2023 12:30PM  256620        10  0086354607   
7  Feb  3 2023 12:30PM  256620        10  0086354604   
8  Feb  3 2023 12:30PM  256620        10  0086354627   
9  Feb  3 2023 12:30PM  256620        10  0086354623   

                         Customer ShipmentStatus  
0               NBTY Global, Inc.       Released  
1  Sartorius Bioprocess Solutions       Released  
2               ISDIN Corporation       Released  
3               ISDIN Corporation       Released  
4               ISDIN Corporation       Released  
5               ISDIN Corporation       Released  
6               ISDIN Corporation       Released  
7               ISDIN Corporation       Released  
8               ISDIN Corporation       Released  
9               ISDIN Corporation       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
38  256618       Released          5
39  256619       Released          2
40  256620       Released         21
41  256621       Released          1
42  256623       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
256618                NaN        NaN       5.0
256619                NaN        NaN       2.0
256620                NaN        NaN      21.0
256621                NaN        NaN       1.0
256623                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
256551                0.0        0.0       3.0
256556                0.0        1.0       0.0
256580                0.0        0.0       1.0
256583                2.0       12.0       1.0
256585                1.0        3.0      18.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
256551                  0          0         3
256556                  0          1         0
256580                  0          0         1
256583                  2         12         1
256585                  1          3        18

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               256551          0          0         3
1               256556          0          1         0
2               256580          0          0         1
3               256583          2         12         1
4               256585          1          3        18

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               256551                            3
1               256556                   1         
2               256580                            1
3               256583         2        12        1
4               256585         1         3       18

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                        Customer
0   Feb  3 2023 12:40PM  256623        12               NBTY Global, Inc.
1   Feb  3 2023 12:35PM  256621        16  Sartorius Bioprocess Solutions
2   Feb  3 2023 12:30PM  256620        10               ISDIN Corporation
23  Feb  3 2023 12:28PM  256619        10                     Emerginnova
25  Feb  3 2023 12:25PM  256618        10               ISDIN Corporation
30  Feb  3 2023 11:51AM  256615        10               ISDIN Corporation
36  Feb  3 2023 11:19AM  256614        19             AdvaGen Pharma, Ltd
57  Feb  3 2023 11:09AM  256613        19             AdvaGen Pharma, Ltd
58  Feb  3 2023 10:41AM  256610        19             AdvaGen Pharma, Ltd
75  Feb  3 2023 10:40AM  256609        10                Eywa Pharma Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                        Customer  \
0  Feb  3 2023 12:40PM  256623        12               NBTY Global, Inc.   
1  Feb  3 2023 12:35PM  256621        16  Sartorius Bioprocess Solutions   
2  Feb  3 2023 12:30PM  256620        10               ISDIN Corporation   
3  Feb  3 2023 12:28PM  256619        10                     Emerginnova   
4  Feb  3 2023 12:25PM  256618        10               ISDIN Corporation   
5  Feb  3 2023 11:51AM  256615        10               ISDIN Corporation   
6  Feb  3 2023 11:19AM  256614        19             AdvaGen Pharma, Ltd   
7  Feb  3 2023 11:09AM  256613        19             AdvaGen Pharma, Ltd   
8  Feb  3 2023 10:41AM  256610        19             AdvaGen Pharma, Ltd   
9  Feb  3 2023 10:40AM  256609        10                Eywa Pharma Inc.   

  Completed Executing Released  
0                            1  
1                            1  
2                           21  
3                            2  
4                            5  
5                            6  
6                  14        7  
7                            1  
8                   1       16  
9                   6       10

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                        Customer  \
0  Feb  3 2023 12:40PM  256623        12               NBTY Global, Inc.   
1  Feb  3 2023 12:35PM  256621        16  Sartorius Bioprocess Solutions   
2  Feb  3 2023 12:30PM  256620        10               ISDIN Corporation   
3  Feb  3 2023 12:28PM  256619        10                     Emerginnova   
4  Feb  3 2023 12:25PM  256618        10               ISDIN Corporation   
5  Feb  3 2023 11:51AM  256615        10               ISDIN Corporation   
6  Feb  3 2023 11:19AM  256614        19             AdvaGen Pharma, Ltd   
7  Feb  3 2023 11:09AM  256613        19             AdvaGen Pharma, Ltd   
8  Feb  3 2023 10:41AM  256610        19             AdvaGen Pharma, Ltd   
9  Feb  3 2023 10:40AM  256609        10                Eywa Pharma Inc.   

  Released Executing Completed  
0        1                      
1        1                      
2       21                      
3        2                      
4        5                      
5        6                      
6        7        14            
7        1                      
8       16         1            
9       10         6

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Feb  3 2023 12:40PM  256623        12               NBTY Global, Inc.   
1  Feb  3 2023 12:35PM  256621        16  Sartorius Bioprocess Solutions   
2  Feb  3 2023 12:30PM  256620        10               ISDIN Corporation   
3  Feb  3 2023 12:28PM  256619        10                     Emerginnova   
4  Feb  3 2023 12:25PM  256618        10               ISDIN Corporation   

  Released Executing Completed  
0        1                      
1        1                      
2       21                      
3        2                      
4        5

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Feb  3 2023 12:40PM  256623        12               NBTY Global, Inc.   
1  Feb  3 2023 12:35PM  256621        16  Sartorius Bioprocess Solutions   
2  Feb  3 2023 12:30PM  256620        10               ISDIN Corporation   
3  Feb  3 2023 12:28PM  256619        10                     Emerginnova   
4  Feb  3 2023 12:25PM  256618        10               ISDIN Corporation   

   Released  Executing  Completed  
0       1.0        NaN        NaN  
1       1.0        NaN        NaN  
2      21.0        NaN        NaN  
3       2.0        NaN        NaN  
4       5.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Feb  3 2023 12:40PM  256623        12               NBTY Global, Inc.   
1  Feb  3 2023 12:35PM  256621        16  Sartorius Bioprocess Solutions   
2  Feb  3 2023 12:30PM  256620        10               ISDIN Corporation   
3  Feb  3 2023 12:28PM  256619        10                     Emerginnova   
4  Feb  3 2023 12:25PM  256618        10               ISDIN Corporation   

   Released  Executing  Completed  
0       1.0        0.0        0.0  
1       1.0        0.0        0.0  
2      21.0        0.0        0.0  
3       2.0        0.0        0.0  
4       5.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         2565977      67.0       13.0        1.0
12          256623       1.0        0.0        0.0
15         1539587      26.0       33.0        9.0
16          513207       1.0        1.0        0.0
19         1026420      25.0       27.0        2.0
21          769800       3.0        0.0        0.0
22          513187       2.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  2565977      67.0       13.0        1.0
1        12   256623       1.0        0.0        0.0
2        15  1539587      26.0       33.0        9.0
3        16   513207       1.0        1.0        0.0
4        19  1026420      25.0       27.0        2.0
5        21   769800       3.0        0.0        0.0
6        22   513187       2.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      67.0       13.0        1.0
1        12       1.0        0.0        0.0
2        15      26.0       33.0        9.0
3        16       1.0        1.0        0.0
4        19      25.0       27.0        2.0
5        21       3.0        0.0        0.0
6        22       2.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   67.0
1        12  Released    1.0
2        15  Released   26.0
3        16  Released    1.0
4        19  Released   25.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12    15   16    19   21   22
Status                                         
Completed   1.0  0.0   9.0  0.0   2.0  0.0  0.0
Executing  13.0  0.0  33.0  1.0  27.0  0.0  0.0
Released   67.0  1.0  26.0  1.0  25.0  3.0  2.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12    15   16    19   21   22
0          Completed   1.0  0.0   9.0  0.0   2.0  0.0  0.0
1          Executing  13.0  0.0  33.0  1.0  27.0  0.0  0.0
2           Released  67.0  1.0  26.0  1.0  25.0  3.0  2.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12    15   16    19   21   22
0  Completed   1.0  0.0   9.0  0.0   2.0  0.0  0.0
1  Executing  13.0  0.0  33.0  1.0  27.0  0.0  0.0
2   Released  67.0  1.0  26.0  1.0  25.0  3.0  2.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()